In [ ]:
import pandas as pd
import time
import random
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# --- CONFIG ---
KEYWORD = "Sekolah di Banyuwangi"
SCROLL_COUNT = 30 
OUTPUT_FILE = "database_sekolah_banyuwangi_lengkap_new.xlsx"

chrome_options = Options()
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])

# Optimasi: Matikan gambar agar loading lebih cepat
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.maximize_window()

def smart_sleep(min_sec, max_sec):
    time.sleep(random.uniform(min_sec, max_sec))

def get_detail_info():
    """Fungsi mengambil Alamat, Website, dan Telepon dari panel detail Maps"""
    data = {
        "alamat": "Tidak ditemukan",
        "website": "Tidak ada",
        "telepon": "Tidak ada"
    }
    
    try:
        # 1. Ambil Alamat (Mencari button dengan icon address)
        try:
            addr_elem = driver.find_element(By.CSS_SELECTOR, "button[data-item-id='address'] div.fontBodyMedium")
            data["alamat"] = addr_elem.text.strip()
        except: pass

        # 2. Ambil Website
        try:
            web_elem = driver.find_element(By.CSS_SELECTOR, "a[data-item-id='authority']")
            data["website"] = web_elem.get_attribute("href")
        except: pass

        # 3. Ambil Telepon
        try:
            tel_elem = driver.find_element(By.CSS_SELECTOR, "button[data-item-id*='phone:tel:'] div.fontBodyMedium")
            data["telepon"] = tel_elem.text.strip()
        except: pass
            
    except Exception as e:
        print(f"Error extracting details: {e}")
    
    return data

print(f"Memulai Scraping: {KEYWORD}")
print("=" * 60)

try:
    # Buka Google Maps dengan keyword pencarian
    driver.get(f"https://www.google.com/maps/search/{KEYWORD.replace(' ', '+')}")
    smart_sleep(5, 7)

    # 1. Proses Scrolling untuk load daftar di sidebar
    print("Scrolling sidebar...")
    for i in range(SCROLL_COUNT):
        try:
            panel = driver.find_element(By.XPATH, '//div[@role="feed"]')
            driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', panel)
            smart_sleep(1.5, 2.5)
            
            # Cek jika sudah mentok akhir daftar
            if "reached the end" in driver.page_source.lower() or "akhir daftar" in driver.page_source.lower():
                break
        except: break

    # 2. Kumpulkan Link Sekolah
    links = driver.find_elements(By.CSS_SELECTOR, "a[href*='maps/place']")
    school_list = []
    seen_urls = set()

    for link in links:
        url = link.get_attribute("href")
        nama = link.get_attribute("aria-label")
        if url and nama and url not in seen_urls:
            school_list.append({"nama": nama, "url": url})
            seen_urls.add(url)

    print(f"Ditemukan {len(school_list)} sekolah. Mulai mengambil detail...")

    # 3. Ambil Detail Satu Per Satu
    all_results = []
    for idx, school in enumerate(school_list):
        print(f"[{idx+1}/{len(school_list)}] {school['nama']}")
        
        try:
            driver.get(school['url'])
            smart_sleep(3, 4) # Tunggu panel detail terbuka sempurna
            
            details = get_detail_info()
            
            all_results.append({
                "Nama Sekolah": school['nama'],
                "Alamat": details['alamat'],
                "Website": details['website'],
                "Telepon": details['telepon'],
                "Google Maps Link": school['url']
            })
            
        except Exception as e:
            print(f"Gagal ambil detail {school['nama']}: {e}")

    # 4. Simpan ke Excel
    if all_results:
        df = pd.DataFrame(all_results)
        df.to_excel(OUTPUT_FILE, index=False)
        print("\n" + "=" * 60)
        print(f"BERHASIL! Data disimpan di: {OUTPUT_FILE}")
        print(f"Total: {len(df)} sekolah")
    else:
        print("Tidak ada data yang berhasil diambil.")

except Exception as e:
    print(f"FATAL ERROR: {e}")

finally:
    driver.quit()

🚀 Memulai Scraping: Sekolahan di Banyuwangi
Scrolling untuk load semua data...
   [1/60] Scrolling...
   [2/60] Scrolling...
   [3/60] Scrolling...
   [4/60] Scrolling...
   [5/60] Scrolling...
   [6/60] Scrolling...
   [7/60] Scrolling...
   [8/60] Scrolling...
   [9/60] Scrolling...
   [10/60] Scrolling...
   [11/60] Scrolling...
   [12/60] Scrolling...
   [13/60] Scrolling...
   [14/60] Scrolling...
   [15/60] Scrolling...
   [16/60] Scrolling...
   [17/60] Scrolling...
Sudah sampai akhir daftar

Mencari semua sekolah di list...
Ditemukan 120 sekolah potensial
Berhasil kumpulkan 120 unique sekolah


[1/120] SMA Negeri 1 Genteng
Alamat: J4WW+WHW, Jl. KH. Wahid Hasyim No.20, Dusun Kopen, Genteng Kulon, Kec. Genteng, Kabupaten Banyuwangi, Jawa Timur 68465
Rating: N/A
Data berhasil tersimpan

[2/120] SMP Ibrahimy Genteng
Alamat: Curahketangi Timur, Setail, Kec. Genteng, Kabupaten Banyuwangi, Jawa Timur 68465
Rating: N/A
Data berhasil tersimpan

[3/120] MI Salafiyah 2 Setail
Alamat: Jl. 

KeyboardInterrupt: 